## 实验2.1红酒质量数据回归探索

---

红酒质量数据集同样来自于 UCI 数据集网站。首先，挑战导入所需模块。

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体（解决中文无法显示的问题）
matplotlib.rcParams['axes.unicode_minus'] = False # 解决保存图像时负号“-”显示方块的问题

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

读取并预览数据集，同时查看数据集列属性。

In [4]:
data = pd.read_csv('winequality-white.csv', sep=';')

In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


下面，将数据集按 7:3 分割成训练集和测试集，设置 `random_state=17`，同时使用 `StandardScaler` 对特征数据规范化。

In [7]:
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3,
                                                          random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

### 线性回归

使用普通最小二乘法线性回归训练一个回归预测模型。

In [8]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

<i class="fa fa-question-circle" aria-hidden="true"> 问题4：</i>训练数据和测试数据上的平均绝对误差 MSE 值是多少？

In [9]:
print("MSE_train: {}" .format(mean_squared_error(y_train, linreg.predict(X_train_scaled))))
print("MSE_test: {}" .format(mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled))))

MSE_train: 0.558060648980358
MSE_test: 0.584247310240454


接下来，你需要将回归系数按绝对值从大到小进行排序。值得注意的是，特征所对应的拟合系数绝对值越大，即代表该特征对目标值的影响越大。这里建议你使用 `pandas.DataFrame` 对数据进行展示。

<i class="fa fa-question-circle" aria-hidden="true"> 问题5：</i>该线性回归模型中，哪些特征对目标值的影响更大？

In [10]:
data.columns.values

array(['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality'], dtype=object)

In [11]:
linreg_coef = pd.DataFrame({'coefficient': linreg.coef_,
                            'coefficient_abs': np.abs(linreg.coef_)},
                              index=data.columns.values[:-1])

In [12]:
linreg_coef

,coefficient,coefficient_abs
fixed acidity,0.097822,0.097822
volatile acidity,-0.192260,0.192260
citric acid,-0.000183,0.000183
residual sugar,0.538164,0.538164
chlorides,0.008127,0.008127
free sulfur dioxide,0.042180,0.042180
total sulfur dioxide,0.014304,0.014304
density,-0.665720,0.665720
pH,0.150036,0.150036
sulphates,0.062053,0.062053


In [13]:
linreg_coef.sort_values(by='coefficient_abs', ascending=False)

,coefficient,coefficient_abs
density,-0.665720,0.665720
residual sugar,0.538164,0.538164
volatile acidity,-0.192260,0.192260
pH,0.150036,0.150036
alcohol,0.129533,0.129533
fixed acidity,0.097822,0.097822
sulphates,0.062053,0.062053
free sulfur dioxide,0.042180,0.042180
total sulfur dioxide,0.014304,0.014304
chlorides,0.008127,0.008127


### Lasso 回归

使用规范化后的数据训练一下 Lasso 回归模型，设置 $\alpha = 0.01$，且 `random_state=17`。

In [14]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

<i class="fa fa-question-circle" aria-hidden="true"> 问题6：</i> 该 Lasso 回归模型中，哪些特征对目标值的影响最小？

In [15]:
lasso1_coef = pd.DataFrame({'coefficient': lasso1.coef_,
                            'coefficient_abs': np.abs(lasso1.coef_)},
                              index=data.columns.values[:-1])

In [16]:
lasso1_coef.sort_values(by='coefficient_abs', ascending=False)

,coefficient,coefficient_abs
alcohol,0.322425,0.322425
residual sugar,0.256363,0.256363
density,-0.235492,0.235492
volatile acidity,-0.188479,0.188479
pH,0.067277,0.067277
free sulfur dioxide,0.043088,0.043088
sulphates,0.029722,0.029722
chlorides,-0.002747,0.002747
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000


接下来，使用 LassoCV 来寻找最合适的 $\alpha$ 参数。

In [17]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(random_state=17, cv=5, alphas=alphas)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [18]:
lasso_cv.alpha_

0.0002833096101839324

<i class="fa fa-question-circle" aria-hidden="true"> 问题7：</i> 上面调参得到的 Lasso 回归模型中，哪些特征对目标值的影响最小？

In [19]:
lasso1_cv_coef = pd.DataFrame({'coefficient': lasso1.coef_,
                            'coefficient_abs': np.abs(lasso1.coef_)},
                              index=data.columns.values[:-1])
lasso1_cv_coef.sort_values(by='coefficient_abs', ascending=False)

,coefficient,coefficient_abs
alcohol,0.322425,0.322425
residual sugar,0.256363,0.256363
density,-0.235492,0.235492
volatile acidity,-0.188479,0.188479
pH,0.067277,0.067277
free sulfur dioxide,0.043088,0.043088
sulphates,0.029722,0.029722
chlorides,-0.002747,0.002747
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000


<i class="fa fa-question-circle" aria-hidden="true"> 问题8：</i> 调参得到的 Lasso 回归模型中，训练数据和测试数据上的平均绝对误差 MSE 值是多少？

In [20]:
print("MSE_train: {}" .format(mean_squared_error(y_train, lasso_cv.predict(X_train_scaled))))
print("MSE_test: {}" .format(mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled))))

MSE_train: 0.5580700141873788
MSE_test: 0.5832976077860629


---